In [1]:
import codecs
import os
from collections import OrderedDict,defaultdict,Counter
import time
import numpy as np
from pandas import Series, DataFrame 

input_path = 'C:\\Users\\user\\Desktop\\RF result\\顯著特徵尋找\\特徵值\\'
vector_path = 'C:\\Users\\user\\Desktop\\RF result\\vector\\'
SC_feature_path = "D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\平衡語料庫\\" #平衡語料庫語言特徵
FC_feature_path = "D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\自由中國\\" #自由中國語言特徵
lei_feature_path = "D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\雷震文本\\" #雷震文本語言特徵

classification_name = ['雷震','殷海光','夏道平','傅正','龍平甫','蔣勻田','朱伴耘','胡適','羅鴻詔']
classification_topic = ['社論','文章','日記']
feature_condicate = ['高頻','bigram','trigram','標點','N+N','N+V','VH+N','D+V','否定','程度','情態']


#建立作者索引，提取各作者文章內容及索引
author_index = []
for index,name in enumerate(classification_name): #建立作者索引
    author_index.append((name,index))
author_index = OrderedDict(author_index) #作者索引排序(依文本數量高到低)

topic_index = []
for index,name in enumerate(classification_topic): #建立作者索引
    topic_index.append((name,index))
topic_index = OrderedDict(topic_index) #作者索引排序(依文本數量高到低)

In [2]:
#輸入介面
def find_input(find):
    
    feature_file_path = '' #符合尋找的特徵檔案路徑
    feature_file_name = '' #符合尋找的特徵檔案名稱
    condicate_label = '' #候選類別名稱
    condicate_index = '' #候選類別索引

    temp = find.split()
    
    if len(temp) != 3:
        print ('請輸入正確值')
        return False
    
    if temp[0] == 'SC':
        feature_file_path = SC_feature_path
    elif temp[0] == 'FC':
        feature_file_path = FC_feature_path
    elif temp[0] == 'lei':
        feature_file_path = lei_feature_path
    else:
        print ('母體選項不符合')
        return False
    
    feature_file_name = [file for file in os.listdir(feature_file_path) if temp[1] in file]
    if len(feature_file_name) == 0:
        print ('輸入的語言特徵不在範圍內')
        return False
    feature_file_name = feature_file_name[0]

    if temp[2] == 'name':
        condicate_label = classification_name
        condicate_index = author_index
    elif temp[2] == 'topic':
        condicate_label = classification_topic
        condicate_index = topic_index
    else:
        print ('領域選項不符合')
        return False
        
    return feature_file_path+feature_file_name,condicate_label,condicate_index

In [6]:
#找出各類別重點詞組
#觀察獨特性、鄰近性、群體性及特殊關係(gini特別高)
a_threshold = 0.011
b_threshold = 0.0027

all_gini_value = []
all_class_feature = defaultdict(list)

for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
        
        if file.split('.')[0].split()[1] not in feature_condicate:
            continue
            
        print (file.split('.')[0])
        #print ('詞組','類別','分數','平均','標準差','數值0次數','總合','類別數據')
        
        feature_path,condicate_label,condicate_index = find_input(file.split('.')[0])
        
        feature_dict = {} #特徵
        feature = []
        with codecs.open(feature_path,'rb','utf8') as ff: #抓取基準特徵
            count = 0
            for i in ff.readlines():
                if '\ufeff' in i: #去掉開頭BOM
                    i = i.replace('\ufeff','')
                if i.strip() != '':
                    feature.append(i.strip().split(',')[0])
                    feature_dict[i.strip().split(',')[0]] = count
                    count += 1
        
        class_vector = defaultdict(list) #各類別文本向量
        with codecs.open(vector_path+file.split('.')[0]+'.csv','rb','utf8') as ff:
            for i in ff.readlines():
                if '\ufeff' in i: #去掉開頭BOM
                    i = i.replace('\ufeff','')
                if i.strip() != '':
                    i = i.strip().split(',')
                    class_vector[condicate_label[int(i[0])]].append([float(j) for j in i[1:]])
        
        content = f.readlines()
        
        feature_importance = defaultdict(list)
        feature_appear = defaultdict(list)
        name = ''
        count = 2
        oob = 0
        for line in content:
            if line[0] == '#':
                name = line[1:].strip()
            #elif name != '' and '雷震' not in name:
            #    continue
            elif count > 0:
                if count == 1:
                    oob = float(line.strip().split(':')[1])
                count = count - 1
            elif line.strip() == '':
                count = 2
            else:
                temp_line = line.strip().split()
                if len(temp_line) == 2:# and float(temp_line[1]) >= 0.01:
                    feature_importance[name].append((temp_line[0],float(temp_line[1])))
                    feature_appear[temp_line[0]].append((name,float(temp_line[1]),oob))
                    all_gini_value.append(float(temp_line[1]))
        
        feature_gini_pair = {}
        for i in feature:
            vector_space = np.zeros((len(condicate_label),len(condicate_label)),np.float64)
            for x in condicate_label:
                for y in condicate_label:
                    if x != y and i in feature_appear:
                        class_check = True
                        for z in feature_appear[i]:
                            if x in z[0] and y in z[0]:
                                vector_space[condicate_index[x],condicate_index[y]] = z[1]#*(1-z[2])
                                #all_gini_value.append(z[1]*(1-z[2]))
                                class_check = False
                                
                        if class_check:
                            vector_space[condicate_index[x],condicate_index[y]] = -1.0
            if i in feature_appear:                       
                feature_gini_pair[i] = vector_space
        
        #特徵觀察
        '''for i in feature:
            for index,j in enumerate(feature_gini_pair[i]):
                if Counter(j.tolist())[0.0] == 1:
                    print (i,condicate_label[index],j)'''
                    
        for i in feature:
            if i not in feature_gini_pair:
                continue
            #分群
            temp_group = []
            for a in range(len(condicate_label)):
                all_check = True
                wait_append = []
                for b in temp_group:
                    if type(b) == int and  0.0 <= feature_gini_pair[i][a,b] < b_threshold:
                        temp_group.remove(b)
                        wait_append.append([b,a])
                        all_check = False
                    elif type(b) == list:
                        check = True
                        for c in b:
                            if feature_gini_pair[i][a,c] >= b_threshold or feature_gini_pair[i][a,c] < 0.0:
                                check = False
                                break
                        if check:
                            b.append(a)
                            all_check = False
                    if not all_check:
                        break
                for d in wait_append:
                    temp_group.append(d)
                if all_check:
                    temp_group.append(a)
                    
            #找出獨特群
            temper = []
            for e in temp_group:
                if type(e) == int:
                    check = True
                    for a in range(len(condicate_label)):
                        if e != a:
                            if feature_gini_pair[i][e,a] < a_threshold:
                                check = False
                                break
                    if check:
                        #temp_group.remove(e)
                        #temp_group.append([e])
                        temper.append([e])
                    else:
                        temper.append(e)
                else:
                    temper.append(e)
                        
            temp_group= temper
            
            #確保群內相似，群外不相似
            temper = []
            for e in temp_group:
                if type(e) == list:
                    check = True
                    for a in range(len(condicate_label)):
                        if a not in e:
                            for t in e:
                                if feature_gini_pair[i][t,a] < a_threshold:
                                    check = False
                                    break
                        if not check:
                            break
                    if not check:
                        #temp_group.remove(e)
                        for t in e:
                            #temp_group.append(t)
                            temper.append(t)
                    else:
                        temper.append(e)
                else:
                    temper.append(e)
                    
            temp_group= temper
        
            #print (feature_gini_pair[i])
            #print (i,temp_group)
            class_group = []
            class_group2 = []
            for e in temp_group:
                if type(e) == list:
                    class_group.append([condicate_label[j] for j in e])
                    #class_group2.append([np.mean(np.array(class_vector[condicate_label[j]]), axis=0)[feature_dict[i]] for j in e])
            if len(class_group) != 0:
                #print (feature_gini_pair[i])
                print (i,temp_group)
                #print (i,class_group)
                #print (class_group2)
                for j in class_group:
                    all_class_feature[' '.join(j)].append(i)
                
            #time.sleep(0.5)
        
        '''feature_importance = defaultdict(list)
        name = ''
        oob = 0
        count = 2
        for line in content:
            if line[0] == '#':
                name = line[1:].strip()
            #elif name != '' and '雷震' not in name:
            #    continue
            elif count > 0:
                if count == 1:
                    oob = float(line.strip().split(':')[1])
                count = count - 1
            elif line.strip() == '':
                count = 2
            else:
                temp_line = line.strip().split()
                if len(temp_line) == 2:# and float(temp_line[1]) > 0.01:
                    feature_importance[name].append((temp_line[0],float(temp_line[1]),oob))
                    
        #詞組 : [兩兩類別,排序,切割分數,
        # (A類別文本該詞組數值最大、最小、平均、標準差、數值0次數、綜合),(B類別文本該詞組數值最大、最小、平均、標準差、數值0次數、綜合)]
        feature_appear = defaultdict(list) 
        for k,v in feature_importance.items():
            for index,e in enumerate(v):
                a_class = np.array(class_vector[k.split()[0]]) 
                b_class = np.array(class_vector[k.split()[1]])
                
                def class_caculator(c): 
                    class_max = max(c[:,feature_dict[e[0]]])
                    class_min = min(c[:,feature_dict[e[0]]])
                    class_mean = np.mean(c, axis=0)[feature_dict[e[0]]]
                    class_std = np.std(c, axis=0)[feature_dict[e[0]]]
                    class_zero = Counter(c[:,feature_dict[e[0]]].tolist())[0]/c.shape[0]
                    class_sum = sum(c[:,feature_dict[e[0]]])
                    
                    return (class_max,class_min,class_mean,class_std,class_zero,class_sum)
                
                a_class = class_caculator(a_class)
                b_class = class_caculator(b_class)
                
                a_b_zero = 0
                if a_class[4] != 0 and b_class[4] != 0:
                    a_b_zero = a_class[4] * b_class[4]
                elif a_class[4] == 0:
                    if b_class[4] != 0:
                        a_b_zero = b_class[4]
                    else:
                        a_b_zero = 0.0000000000001
                else:
                    a_b_zero = a_class[4]
                
                #排名分數 = 排名 * (分割數值/0.01)倒數 * 1-oob倒數 * 兩兩0數值比例倒數   數值越小越好 
                ranking_value = index 
                if e[1]/0.01 == 0:
                    ranking_value = 100/((1-e[2])*a_b_zero)
                else:
                    ranking_value = index/((e[1]/0.01)*(1-e[2])*a_b_zero)
                    
                feature_appear[e[0]].append([k,ranking_value,e[1],a_class,b_class])
                
        for i in feature:
            #if len(feature_appear[i]) == (len(condicate_label)*(len(condicate_label)-1))/2: 
            #    print (i)
            #print (len(feature_appear[i]),condicate_label)
            #print (i,feature_appear[i])
            #print (i,len(feature_appear[i]),sum([j[1] for j in feature_appear[i]]))
            for j in condicate_label:
                temp_check = []
                for k in feature_appear[i]:
                    if j in k[0]:
                        temp_check.append(k)
                #if len(temp_check) == len(condicate_label)-1 and sum([k[1] for k in temp_check]) < 30:
                if len(temp_check) >= len(condicate_label)-2 and sum([k[1] for k in temp_check]) < 30:
                    compare = len(temp_check)
                    def compare_value(com,num):
                        compare = com
                        for l in temp_check:
                            if l[0].split()[0] == j:
                                if l[3][num] > l[4][num]:
                                    compare -= 1
                            elif l[0].split()[1] == j:
                                if l[3][num] < l[4][num]:
                                    compare -= 1
                        return compare
                    
                    class_value = ''
                    if temp_check[0][0].split()[0] == j:
                        class_value = temp_check[0][3]
                    elif temp_check[0][0].split()[1] == j:
                        class_value = temp_check[0][4]
                        
                    #if class_value[4] > 0.5:
                    #    continue
                        
                    print (i,j,sum([k[1] for k in temp_check]),compare_value(compare,2),compare_value(compare,3)
                           ,compare_value(compare,4),compare_value(compare,5),class_value)
                    
                    vector_space = np.zeros((len(condicate_label),len(condicate_label)),np.float64)
                    for x in condicate_label:
                        for y in condicate_label:
                            if x != y:
                                for z in feature_appear[i]:
                                    if x in z[0] and y in z[0]:
                                        vector_space[condicate_index[x],condicate_index[y]] = z[2]
                    print (vector_space[condicate_index[j]])'''
        
        print ()
        #time.sleep(3)
'''with codecs.open('C:\\Users\\user\\Desktop\\all_gini.txt','wb','utf8') as g:
    so = sorted(all_gini_value, reverse = True)
    for i in so:
        g.write(str(i)+'\r\n')'''
for k,v in all_class_feature.items():
    print (k,len(v))
print ('END')

FC bigram name
這種 [0, [1], 2, 3, 4, 5, 6, 7, 8]
的一 [0, 2, 1, 3, 5, 4, 6, [7], 8]
他的 [0, 1, 3, 4, 2, 5, 6, [7], 8]
美國的 [0, 1, 2, 7, 3, 4, 5, [6], 8]
的事 [[1], 2, 0, 4, 5, 8, 3, 6, 7]
事實上 [0, [3], 4, 1, 6, 5, 7, 2, 8]
我的 [0, 2, 3, 8, 1, 4, 5, 6, [7]]
民主國家 [[0], 2, 3, 7, 1, 4, 5, 6, 8]
的辦法 [0, 1, 8, 2, 3, 4, 5, [6], 7]

FC D+V name
所(D)說(VE) [[0], [1], [2], [3], 4, [5], 6, 7, 8]
最(Dfa)大(VH) [[0], [1], [2], [3], [4], 5, 6, [7], 8]
不(D)知(VK) [[0], 1, [2], 3, [4], [5], [6], 7, [8]]
最(Dfa)高(VH) [[0], 2, 1, 3, [4], 5, 6, 7, 8]
可以(D)說(VE) [[0], [1], 2, 3, 4, [5], 6, [7], [8]]
不(D)敢(VL) [[0], [1], [2], [3], [4], [5], [6], [7], [8]]
不(D)願(VK) [0, [1], 2, [3], [4], [5], [6], 7, 8]
很(Dfa)大(VH) [0, 1, 2, 3, [4], 5, 7, 6, 8]
可(D)說(VE) [[0], 1, 2, 3, 4, [5], 6, 7, [8]]
不(D)知道(VK) [0, 2, [3], 4, 1, 5, 6, 7, 8]
最(Dfa)重要(VH) [[0], 1, 3, 4, 2, 6, 7, 5, 8]
又(D)說(VE) [1, 2, 3, 4, [5], 0, 6, [7], 8]
可(D)言(VE) [0, [1], 3, 2, 4, 5, 6, 7, 8]
不(D)出(VC) [[1], 2, 0, 3, 5, 6, 7, 4, 8]
不(D)難(VH) [1, 2, [3], 0, 4, 6, 

In [112]:
#新版，直接觀察各類別
#找出各類別重點詞組
#觀察獨特性、鄰近性、群體性及特殊關係(gini特別高)

label_gini = defaultdict(dict)

for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
            
        #print (file.split('.')[0])
        
        feature_path,condicate_label,condicate_index = find_input(file.split('.')[0])
        
        feature_dict = {} #特徵
        feature = []
        with codecs.open(feature_path,'rb','utf8') as ff: #抓取基準特徵
            count = 0
            for i in ff.readlines():
                if '\ufeff' in i: #去掉開頭BOM
                    i = i.replace('\ufeff','')
                if i.strip() != '':
                    feature.append(i.strip().split(',')[0])
                    feature_dict[i.strip().split(',')[0]] = count
                    count += 1
        
        class_vector = defaultdict(list) #各類別文本向量
        with codecs.open(vector_path+file.split('.')[0]+'.csv','rb','utf8') as ff:
            for i in ff.readlines():
                if '\ufeff' in i: #去掉開頭BOM
                    i = i.replace('\ufeff','')
                if i.strip() != '':
                    i = i.strip().split(',')
                    class_vector[condicate_label[int(i[0])]].append([float(j) for j in i[1:]])
        
        content = f.readlines()
        
        feature_importance = defaultdict(list)
        feature_appear = defaultdict(list)
        name = ''
        count = 2
        oob = 0
        for line in content:
            if line[0] == '#':
                name = line[1:].strip()
            #elif name != '' and '雷震' not in name:
            #    continue
            elif count > 0:
                if count == 1:
                    oob = float(line.strip().split(':')[1])
                count = count - 1
            elif line.strip() == '':
                count = 2
            else:
                temp_line = line.strip().split()
                if len(temp_line) == 2:
                    feature_importance[name].append((temp_line[0],float(temp_line[1])))
                    feature_appear[temp_line[0]].append((name,float(temp_line[1]),oob))
        
        feature_gini_pair = {}
        for i in feature:
            vector_space = np.zeros((len(condicate_label),len(condicate_label)),np.float64) 
            for x in condicate_label:
                for y in condicate_label:
                    if x != y and i in feature_appear:
                        class_check = True
                        for z in feature_appear[i]:
                            if x in z[0] and y in z[0]:
                                vector_space[condicate_index[x],condicate_index[y]] = z[1]
                                class_check = False
                                
                        if class_check:
                            vector_space[condicate_index[x],condicate_index[y]] = -1.0
                                
                    elif x != y and i not in feature_appear:
                        vector_space[condicate_index[x],condicate_index[y]] = -1.0  
                                    
            feature_gini_pair[i] = vector_space
            
            for j in range(len(condicate_label)):
                temp_list = [x for x in vector_space[j].tolist() if x not in (0.0,-1.0,-2.0)]
                for i in range(Counter(vector_space[j].tolist())[0.0]-1):
                    temp_list.append(0.0)
                temp_mean = 0
                if len(temp_list) != 0:
                    temp_mean = sum(temp_list)/len(temp_list)
                
                if len(vector_space[j])-1-len(temp_list) not in label_gini[condicate_label[j]]:
                    label_gini[condicate_label[j]][len(vector_space[j])-1-len(temp_list)] = \
                        [(temp_mean,vector_space[j].tolist(),i)]
                else:
                    label_gini[condicate_label[j]][len(vector_space[j])-1-len(temp_list)].append( \
                        (temp_mean,vector_space[j].tolist(),i))
                   
        #print ()
        #time.sleep(3)

for k,v in label_gini.items():
    print (k,v[1][0])
        
print ('END')

夏道平 (0.020503786218215286, [0.000891766988988, 0.000748889030412, 0.0, -1.0, 0.0383303384906, 0.00456524759249, 0.0139799514176, 0.000150635636517, 0.0848596743709], '各國')
雷震 (0.022047175724995146, [0.0, -1.0, 0.00169029051218, 0.00113592314978, 0.000644115601506, 0.0211969676498, 0.0300466917976, 0.0982819907955, 0.0013342505686], '我的')
社論 (5.40686140843e-05, [0.0, 5.40686140843e-05, -1.0], '胡先生')
胡適 (0.00582620401165, [0.00115530239101, 0.0169231109161, 0.00884820125423, -1.0, 0.00614114189357, 0.00408935531077, 0.00252912105518, 0.0, 0.00109719526069], '的政治')
日記 (0.000218516182313, [-1.0, 0.000218516182313, 0.0], '一天')
龍平甫 (0.005844359427876572, [0.0212113262483, 0.00259036765764, 0.00291936821627, 0.00086053616744, 0.0, 0.0103658834826, 0.00263683634426, -1.0, 0.000326197878626], '民主政治')
殷海光 (0.0034236902000484284, [0.0084100242345, 0.0, 0.00439007517422, -1.0, 0.00156850961819, 0.00262379659047, 0.00354326889116, 0.00317938187361, 0.000250775018189], '政府的')
傅正 (0.01715833086698693

In [87]:
#計算各類別間的預測及OOB error
for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
        content = f.readlines()
        
        print (file)
        
        temp_label = ''
        temp_index = ''
        if 'name' in file:
            temp_label = classification_name
            temp_index = author_index
        elif 'topic' in file:
            temp_label = classification_topic
            temp_index = topic_index
        
        accuracy = []
        vector_space = np.zeros((len(temp_index)+1,len(temp_index)+1),np.float64) #多平均
        
        for i in range(len(content)-1):
            if content[i][0] == '#':
                #accuracy.append((content[i][1:],float(content[i+1].split('：')[1]))) #accuracy
                accuracy.append((content[i][1:],float(content[i+2].split(':')[1]))) #oob error
                
        for i in accuracy:
            #print (i[0],round(i[1],3))
            vector_space[temp_index[i[0].split()[0]],temp_index[i[0].split()[1]]] = round(i[1],3)
            vector_space[temp_index[i[0].split()[1]],temp_index[i[0].split()[0]]] = round(i[1],3)
            
        for i in range(len(temp_index)):
            vector_space[i,-1] = round(sum(vector_space[i])/(len(temp_index)-1),3)
        for i in range(len(temp_index)):
            vector_space[-1,i] = round(sum(vector_space[:,i])/(len(temp_index)-1),3)
        vector_space[-1,-1] = round(sum(vector_space[-1])/(len(temp_index)),3)
        
        print (' '.join(temp_label+['平均']))
        print (vector_space)
        #df = DataFrame(vector_space,index=temp_label,columns=temp_label)
        print ()

FC bigram name.txt
雷震 殷海光 夏道平 傅正 龍平甫 蔣勻田 朱伴耘 胡適 羅鴻詔 平均
[[ 0.     0.054  0.209  0.163  0.     0.167  0.121  0.102  0.071  0.111]
 [ 0.054  0.     0.077  0.021  0.015  0.034  0.036  0.053  0.     0.036]
 [ 0.209  0.077  0.     0.103  0.     0.029  0.075  0.132  0.015  0.08 ]
 [ 0.163  0.021  0.103  0.     0.016  0.02   0.041  0.04   0.021  0.053]
 [ 0.     0.015  0.     0.016  0.     0.097  0.043  0.042  0.059  0.034]
 [ 0.167  0.034  0.029  0.02   0.097  0.     0.067  0.066  0.069  0.069]
 [ 0.121  0.036  0.075  0.041  0.043  0.067  0.     0.034  0.054  0.059]
 [ 0.102  0.053  0.132  0.04   0.042  0.066  0.034  0.     0.053  0.065]
 [ 0.071  0.     0.015  0.021  0.059  0.069  0.054  0.053  0.     0.043]
 [ 0.111  0.036  0.08   0.053  0.034  0.069  0.059  0.065  0.043  0.061]]

FC D+V name.txt
雷震 殷海光 夏道平 傅正 龍平甫 蔣勻田 朱伴耘 胡適 羅鴻詔 平均
[[ 0.     0.393  0.433  0.204  0.171  0.3    0.121  0.322  0.196  0.268]
 [ 0.393  0.     0.154  0.213  0.118  0.241  0.214  0.158  0.148  0.205]
 [ 0.433  0.154

In [8]:
condicate_author_path = "D:\\課業相關\\論文資料\\論文程式\\condicate\\author\\" #候選作者文本
condicate_topic_path = "D:\\課業相關\\論文資料\\論文程式\\condicate\\topic\\" #候選主題文本

for file in os.listdir(condicate_author_path):
    with codecs.open(condicate_author_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        print (file)
        #反共(VH)戰爭(Na) 自由(VH)中國(Nc)
        for i in range(len(content)-1):
            if content[i] == '獨裁(VH)' and content[i+1] == '者(Na)':
                if i > 4 and len(content) - i > 5:
                    print (''.join(content[i-5:i+6]))
                elif i <= 4 and len(content) - i > 5:
                    print (''.join(content[:i+6]))
                elif i > 4 and len(content) - i <= 5:
                    print (''.join(content[i-5:]))
                else:
                    print (''.join(content))
    print ()
    #time.sleep(0.5)

傅正_1.txt

傅正_10.txt

傅正_11.txt

傅正_12.txt

傅正_13.txt

傅正_14.txt

傅正_15.txt

傅正_16.txt

傅正_17.txt

傅正_18.txt

傅正_19.txt

傅正_2.txt

傅正_20.txt

傅正_3.txt

傅正_4.txt

傅正_5.txt

傅正_6.txt

傅正_7.txt

傅正_8.txt

傅正_9.txt

夏道平_1.txt

夏道平_10.txt

夏道平_11.txt

夏道平_12.txt

夏道平_13.txt

夏道平_14.txt

夏道平_15.txt

夏道平_16.txt

夏道平_17.txt

夏道平_18.txt

夏道平_19.txt

夏道平_2.txt

夏道平_20.txt

夏道平_21.txt

夏道平_22.txt

夏道平_23.txt

夏道平_24.txt

夏道平_25.txt

夏道平_26.txt

夏道平_27.txt

夏道平_28.txt

夏道平_29.txt

夏道平_3.txt

夏道平_30.txt

夏道平_31.txt

夏道平_32.txt

夏道平_33.txt

夏道平_34.txt

夏道平_35.txt

夏道平_36.txt

夏道平_37.txt

夏道平_38.txt

夏道平_4.txt

夏道平_5.txt

夏道平_6.txt

夏道平_7.txt

夏道平_8.txt

夏道平_9.txt

朱伴耘_1.txt

朱伴耘_10.txt

朱伴耘_11.txt

朱伴耘_12.txt

朱伴耘_13.txt

朱伴耘_14.txt

朱伴耘_15.txt

朱伴耘_16.txt

朱伴耘_17.txt

朱伴耘_18.txt

朱伴耘_19.txt
問題(Na)。(PERIODCATEGORY)集權(VH)國家(Na)的(DE)獨裁(VH)者(Na)知道(VK)民主(Na)是(SHI)今日(Nd)

朱伴耘_2.txt

朱伴耘_20.txt

朱伴耘_21.txt

朱伴耘_22.txt

朱伴耘_23.txt

朱伴耘_24.txt
萬(Neu)世(Na)。(PERIODCATEGORY)這(Nep)是(SHI)獨裁(VH)者(Na)騙人(VA)的(DE)文字(

In [84]:
from sklearn.externals import joblib
from sklearn.tree import export_graphviz
import pydotplus

model_path = 'C:\\Users\\user\\Desktop\\RF result\\顯著特徵尋找\\model\\'

def out_tree_image(model,feature,condicate_label,name):
    dot_data = export_graphviz(model.estimators_[-1], out_file=None,
                                filled=True,feature_names=feature,class_names=condicate_label,
                                 proportion=True,rounded=True,special_characters=False) 
    dot_data = dot_data.replace('helvetica','kaiu') #字型調換
    graph = pydotplus.graph_from_dot_data(dot_data)
    graph.write_png("C:\\Users\\user\\Desktop\\tree2\\"+name+".png")

for model in os.listdir(model_path):
    clf = joblib.load(model_path+model)
    print (model.split('.')[0])
    #print (clf.oob_decision_function_)
    #print (clf.oob_score_)
    
    feature_path,condicate_label,condicate_index = find_input(' '.join(model.split('.')[0].split()[:3]))
    temp_label = []
    for i in condicate_label:
        if i in model.split('.')[0].split():
            temp_label.append(i)
        
    feature = []
    with codecs.open(feature_path,'rb','utf8') as ff: #抓取基準特徵
        for i in ff.readlines():
            if '\ufeff' in i: #去掉開頭BOM
                i = i.replace('\ufeff','')
            if i.strip() != '':
                feature.append(i.strip().split(',')[0])
    
    #out_tree_image(clf,feature,temp_label,model.split('.')[0])
    
    print ()
    #time.sleep(0.5)

FC bigram name 傅正 朱伴耘

(dot.exe:11732): Pango-WARNING **: couldn't load font "kaiu Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.


FC bigram name 傅正 羅鴻詔

(dot.exe:12016): Pango-WARNING **: couldn't load font "kaiu Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.


FC bigram name 傅正 胡適

(dot.exe:11012): Pango-WARNING **: couldn't load font "kaiu Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.


FC bigram name 傅正 蔣勻田

(dot.exe:6912): Pango-WARNING **: couldn't load font "kaiu Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.


FC bigram name 傅正 龍平甫

(dot.exe:2064): Pango-WARNING **: couldn't load font "kaiu Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.


FC bigram name 夏道平 傅正

(dot.exe:8236): Pango-WARNING **: couldn't load font "kaiu Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.


FC bigram name 夏道平 朱伴耘

(dot.exe:3764)